In [1]:
from Model.config import config
from Model.codes import Codes
from Model.file import File
from Model.functions import Function
from Model.indicator import Indicator
from Model.stock import Stock
from Model.strategy import Strategy
from functools import partial
import datetime
import itertools
import quandl
import dill as pickle
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
quandl.ApiConfig.api_key = config.API_KEY # mandatory to make api calls on quandl
print(config.TEST)

False


# Stock creation and update

In [2]:
codes = Codes()

In [3]:
stocks = []
for index, row in codes.data[codes.data.INCLUDE_FOR_ANALYSIS].iterrows():
    stocks.append(Stock(index))

In [4]:
len(stocks) # must be 212

212

In [5]:
for stock in stocks:
    stock.refresh()

AALB successfully updated!
AB successfully updated!
ABI successfully updated!
ABN successfully updated!
AC successfully updated!
ACA successfully updated!
AD successfully updated!
AF successfully updated!
AGFB successfully updated!
AGN successfully updated!
AGS successfully updated!
AI successfully updated!
AIR successfully updated!
AKE successfully updated!
AKZA successfully updated!
ALADO successfully updated!
ALAUP successfully updated!
ALBPS successfully updated!
ALCLS successfully updated!
ALCYB successfully updated!
ALD successfully updated!
ALHYG successfully updated!
ALNXT successfully updated!
ALO successfully updated!
ALT successfully updated!
ALTR successfully updated!
ALVMG successfully updated!
AMG successfully updated!
AMUN successfully updated!
APAM successfully updated!
ARCAD successfully updated!
ASM successfully updated!
ASML successfully updated!
ASRNL successfully updated!
ATC successfully updated!
ATCB successfully updated!
ATO successfully updated!
BAMNB successfu

# Indicators

In [9]:
def SMA(df, window, column = 'CLOSE'):
    return df[column].rolling(window = window).mean()
SMA = Function('SMA', SMA)

In [10]:
SMA20 = Indicator('SMA20', partial(SMA.execute, window = 20))
SMA50 = Indicator('SMA50', partial(SMA.execute, window = 50))

In [11]:
for stock in stocks:
    stock.add_indicator(SMA20)
    stock.add_indicator(SMA50)

# Strategy

In [12]:
def buy_condition(stock, row):
    if np.isnan(row.SMA20) or np.isnan(row.SMA50):
        return True
    else:
        return row.SMA20 > row.SMA50

def sell_condition(stock, row, trade):
    return row.SMA20 < row.SMA50

SMA20_SMA50 = Strategy('SMA20_SMA50', buy_condition, sell_condition)

In [13]:
SMA20_SMA50.refresh(stocks)

Trades for strategy SMA20_SMA50 successfully updated.


In [14]:
SMA20_SMA50.summary()

SMA20_SMA50 : 17.14% in 2085 trades.
Average trade duration : 66.56137724550898 days.


TypeError: '>' not supported between instances of 'NoneType' and 'int'

In [20]:
SMA20_SMA50.last_status

{'AALB': 'closed',
 'AB': 'pending',
 'ABI': 'closed',
 'ABN': 'pending',
 'AC': 'closed',
 'ACA': 'pending',
 'AD': 'closed',
 'AF': 'pending',
 'AGFB': 'closed',
 'AGN': 'closed',
 'AGS': 'pending',
 'AI': 'closed',
 'AIR': 'pending',
 'AKE': 'pending',
 'AKZA': 'pending',
 'ALADO': 'pending',
 'ALAUP': 'pending',
 'ALBPS': 'closed',
 'ALCLS': 'closed',
 'ALCYB': 'closed',
 'ALD': 'pending',
 'ALHYG': 'pending',
 'ALNXT': 'closed',
 'ALO': 'closed',
 'ALT': 'closed',
 'ALTR': 'closed',
 'ALVMG': 'closed',
 'AMG': 'closed',
 'AMUN': 'pending',
 'APAM': 'pending',
 'ARCAD': 'pending',
 'ASM': 'closed',
 'ASML': 'closed',
 'ASRNL': 'pending',
 'ATC': 'closed',
 'ATCB': 'closed',
 'ATO': 'closed',
 'BAMNB': 'pending',
 'BEKB': 'closed',
 'BESI': 'closed',
 'BINCK': 'pending',
 'BN': 'pending',
 'BNP': 'closed',
 'BOKA': 'closed',
 'BOL': 'pending',
 'BPI': 'pending',
 'BPOST': 'pending',
 'BRNL': 'closed',
 'BVI': 'closed',
 'CA': 'pending',
 'CAP': 'closed',
 'CATG': 'pending',
 'CCE': 

In [76]:
def load_stocks():
    stocks = []
    codes = Codes()
    for index, _ in codes.data[codes.data.INCLUDE_FOR_ANALYSIS].iterrows():
        stocks.append(File(index, class_type = 'Stock').load())
    return stocks
    
def display_on_graphs(focus, y, figsize = (20,100)):
    fig, axis = plt.subplots(nrows = int(len(focus) / 4) + 1, ncols=4, sharex = False, sharey = False, figsize=figsize)
    plt.subplots_adjust(wspace = 0.1, hspace = 0.2)
    j = 0
    for ax in axis.reshape(-1):
        if j < len(focus):
            stock = focus[j]
            ax.plot(stock.data.index, stock.data[y])
            ax.set_title(stock.name)
            ax.tick_params(axis='both', which='both', bottom=False, labelbottom=False, left=False, labelleft=True)
            j = j+1
    plt.show()

def display_strategies_on_graphs(stocks, strategy, figsize = (20,130)):
    j = 0
    fig, axis = plt.subplots(nrows = int(len(stocks) / 4) + 1, ncols=4, sharex = False, sharey = False, figsize=figsize)
    trades = strategy.to_dataframe()
    for ax in axis.reshape(-1):
        if j < len(stocks):
            stock = stocks[j]
            ax.plot(stock.data.index, stock.data.CLOSE)
            ax.set_title(stock.name, fontsize = 8, y = 0.98)
            df = trades[trades.STOCK_NAME == stock.name]
            for index, row in df.iterrows():
                opening_date = row.OPENING_DATE
                closing_date = row.CLOSING_DATE
                ax.axvline(row.OPENING_DATE, color='grey', linestyle ='dashed', linewidth=0.5)
                if str(closing_date) == 'NaT':
                    closing_date = stock.last_date
                else:
                    ax.axvline(row.CLOSING_DATE, color='grey', linestyle ='dashed', linewidth=0.5)                    
                sub_stock = stock.data.loc[opening_date:closing_date]
                if row.PERFORMANCE >= 1:
                    color = 'green'
                else:
                    color = 'red'
                ax.plot(sub_stock.index, sub_stock.CLOSE, color=color)
        ax.tick_params(axis='both', which='both', bottom=False, labelbottom=False, left=False, labelleft=False)
        j = j + 1
    #plt.show()

In [77]:
stocks = load_stocks()

In [80]:
stocks[0].get_attributes()

name : AALB
type : Stock
test : False
filename : Stock/AALB.p
folder : /Users/hugofayolle/Desktop/Bourse/Bourse_test/Data/
path : /Users/hugofayolle/Desktop/Bourse/Bourse_test/Data/Stock/AALB.p
creation_date : 2018-09-09 17:13:46
last_update : 2018-09-09 17:16:46
data :               OPEN    HIGH     LOW   CLOSE    VOLUME      TURNOVER     SMA20  \
DATE                                                                           
2014-02-14  23.955  24.090  23.875  23.960  112257.0  2.690824e+06       NaN   
2014-02-17  23.990  24.210  23.925  24.120  140913.0  3.396915e+06       NaN   
2014-02-18  24.120  24.120  23.820  23.990  169511.0  4.060882e+06       NaN   
2014-02-19  23.880  23.965  23.710  23.775  143329.0  3.412093e+06       NaN   
2014-02-20  23.610  23.720  23.555  23.665  111246.0  2.630201e+06       NaN   
2014-02-21  23.740  23.820  23.620  23.715  123793.0  2.935698e+06       NaN   
2014-02-24  23.715  23.815  23.625  23.760  132331.0  3.140268e+06       NaN   
2014-02-2

In [56]:
s.l

,OPEN,HIGH,LOW,CLOSE,VOLUME,TURNOVER,SMA20,SMA50
DATE,,,,,,,,
2014-02-14,23.955,24.090,23.875,23.960,112257.0,2690823.825,NaN,NaN
2014-02-17,23.990,24.210,23.925,24.120,140913.0,3396914.560,NaN,NaN
2014-02-18,24.120,24.120,23.820,23.990,169511.0,4060882.350,NaN,NaN
2014-02-19,23.880,23.965,23.710,23.775,143329.0,3412092.760,NaN,NaN
2014-02-20,23.610,23.720,23.555,23.665,111246.0,2630200.745,NaN,NaN
2014-02-21,23.740,23.820,23.620,23.715,123793.0,2935697.820,NaN,NaN
2014-02-24,23.715,23.815,23.625,23.760,132331.0,3140268.225,NaN,NaN
2014-02-25,23.785,23.865,23.640,23.755,168405.0,4000691.910,NaN,NaN


In [39]:
SMA20_SMA50.trades

[2014-06-20 00:00:00	2014-07-11 00:00:00	AALB	closed		24.76	22.7	-8.393%,
 2014-11-24 00:00:00	2015-05-07 00:00:00	AALB	closed		22.49	27.135	20.557%,
 2015-08-06 00:00:00	2015-09-08 00:00:00	AALB	closed		29.245	27.415	-6.332%,
 2015-10-26 00:00:00	2016-01-11 00:00:00	AALB	closed		28.95	28.83	-0.494%,
 2016-03-11 00:00:00	2016-06-27 00:00:00	AALB	closed		29.01	28.0	-3.559%,
 2016-08-11 00:00:00	2016-10-14 00:00:00	AALB	closed		29.915	29.07	-2.902%,
 2016-12-14 00:00:00	2017-06-12 00:00:00	AALB	closed		29.6	35.66	20.377%,
 2017-06-21 00:00:00	2017-06-30 00:00:00	AALB	closed		36.305	34.885	-3.988%,
 2017-08-03 00:00:00	2018-02-12 00:00:00	AALB	closed		36.18	40.24	11.133%,
 2018-03-23 00:00:00	2018-04-20 00:00:00	AALB	closed		41.91	40.87	-2.56%,
 2018-05-29 00:00:00	2018-07-04 00:00:00	AALB	closed		43.17	40.06	-7.278%,
 2014-11-07 00:00:00	2015-04-02 00:00:00	AB	closed		13.28	15.5	16.624%,
 2015-06-03 00:00:00	2015-07-09 00:00:00	AB	closed		16.67	14.01	-16.024%,
 2015-11-17 00:00:00	2016-0